# À propos de ce livret



Recherches liées au développement de cette méthode : 
>Yu, Hsiang-Fu, Nikhil Rao et Inderjit S. Dhillon (2016). [**Temporal regularized matrix factorization for high-dimensional time series prediction**](http://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf).


La méthode implantés provient de ces sources : 
>[**Dépôt 1**](https://github.com/xinychen/transdim) | *Version en python*



**Note sur le cachier**  
- Terminer la présentation des résultats (RMSE, MAPE)
- Définir les variables utilisées

## Préparation préalable à l'utilisation

In [2]:
# Utilités
import os
import numpy as np
from numpy.linalg import inv as inv
import pandas as pd

# Chargement des données
import scipy.io

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook, tqdm

# Présentation des résultats
import matplotlib.pyplot as plt

In [3]:
os.chdir("C:/Users/amass/OneDrive/02_Education/02_Maitrise/Cours/PROJET_MAITRISE/application")

# Données

## Chargement des données

In [4]:
# Chargement du tenseur de données
tensor = scipy.io.loadmat('data/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']

# Chargement de la matrice aléatoire de données
random_matrix = scipy.io.loadmat('data/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

# Chargement du tenseur aléatoire de données
random_tensor = scipy.io.loadmat('data/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

print("Tensor shape : {:} | Random matrix shape : {:} | Random tensor shape : {:}".format(tensor.shape, 
                                                                                        random_matrix.shape,
                                                                                        random_tensor.shape))

Tensor shape : (214, 61, 144) | Random matrix shape : (214, 61) | Random tensor shape : (214, 61, 144)


## Format des données

In [5]:
petit_donnees = True

# Paramètres du jeu de données tronqué
capteurs = 50
jours = 10
sequences = tensor.shape[2]

matrice = True

if petit_donnees == True:
    tensor = tensor[0:capteurs, 0:jours, 0:sequences]
    
    random_tensor = random_tensor[0:capteurs, 0:jours, 0:sequences]
    

print("The tensors shapes are : {:} | {:}".format(tensor.shape, random_tensor.shape))

if matrice == True:
    dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
    
    print("The matrix shape is : {:}".format(dense_mat.shape))

The tensors shapes are : (50, 10, 144) | (50, 10, 144)
The matrix shape is : (50, 1440)


## Scénario de manquants

In [6]:
missing_rate = 0.4

random_missing = True

# Revoir le code pour que les deux choix puissent être respectés sur l'ensemble des matrices/ tenseurs
if random_missing == True:
    print("Random missing (RM) scenario")
    binary_mat = (np.round(random_tensor + 0.5 - missing_rate).reshape([
        random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]
    ]))
    print(("\nBinary matrix shape : {:}".format(binary_mat.shape)))
else:
    print("Non-random missing (NM) scenario")
    binary_tensor = np.zeros(tensor.shape)
    for i1 in range(tensor.shape[0]):
        for i2 in range(tensor.shape[1]):
            binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 -
                                                missing_rate)
    binary_mat = binary_tensor.reshape([
        binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]
    ])
    print(("\nBinary matrix shape : {:}".format(binary_mat.shape)))

sparse_mat = np.multiply(dense_mat, binary_mat)

Random missing (RM) scenario

Binary matrix shape : (50, 1440)


# Partie 2 : Modèle

## Partie 2.1 : Spécification du modèle

In [61]:
def TRMF(dense_mat, sparse_mat, init_para, init_hyper, time_lags, maxiter):
    """
    Temporal Regularized Matrix Factorization, TRMF.
    
    Paramètres en entrée :
        dense_mat
        sparse_mat
        init_para
        init_hyper
        time_lags
        maxiter
    
    
    Paramètres en sortie : 
        mat_hat : Matrice imputée
        mape : MAPe
        rmse : Erreur au carré
    
    
    """
    # Variables bidons pour la présentation des résultats
    lrmse = []
    lmape = []

    ## Initialize parameters
    W = init_para["W"]
    X = init_para["X"]
    theta = init_para["theta"]

    ## Set hyperparameters
    lambda_w = init_hyper["lambda_w"]
    lambda_x = init_hyper["lambda_x"]
    lambda_theta = init_hyper["lambda_theta"]
    eta = init_hyper["eta"]

    dim1, dim2 = sparse_mat.shape
    pos_train = np.where(sparse_mat != 0)
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    binary_mat = sparse_mat.copy()
    binary_mat[pos_train] = 1
    d, rank = theta.shape


    for it in notebook.tqdm(list(range(maxiter)),
                            desc="Itérations",
                            leave=False):
        ## Update spatial matrix W
        for i in notebook.tqdm(list(range(dim1)),
                               desc="Mise à jour W",
                               leave=False):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            vec0 = np.matmul(Xt.T, sparse_mat[i, pos0[0]])
            mat0 = inv(np.matmul(Xt.T, Xt) + lambda_w * np.eye(rank))
            W[i, :] = np.matmul(mat0, vec0)
        ## Update temporal matrix X
        for t in notebook.tqdm(list(range(dim2)),
                               desc="Mise à jour X",
                               leave=False):
            pos0 = np.where(sparse_mat[:, t] != 0)
            Wt = W[pos0[0], :]
            Mt = np.zeros((rank, rank))
            Nt = np.zeros(rank)
            if t < np.max(time_lags):
                Pt = np.zeros((rank, rank))
                Qt = np.zeros(rank)
            else:
                Pt = np.eye(rank)
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(
                        np.where((t + time_lags >= np.max(time_lags))
                                 & (t + time_lags < dim2)))[0]
                for k in index:
                    Ak = theta[k, :]
                    Mt += np.diag(Ak**2)
                    theta0 = theta.copy()
                    theta0[k, :] = 0
                    Nt += np.multiply(
                        Ak, X[t + time_lags[k], :] -
                        np.einsum('ij, ij -> j', theta0,
                                  X[t + time_lags[k] - time_lags, :]))
            vec0 = np.matmul(
                Wt.T, sparse_mat[pos0[0], t]) + lambda_x * Nt + lambda_x * Qt
            mat0 = inv(np.matmul(Wt.T, Wt) + lambda_x * Mt + lambda_x * Pt)
            X[t, :] = np.matmul(mat0, vec0)
        ## Update AR coefficients theta
        for k in notebook.tqdm(list(range(d)),
                               desc="Mise à jour AR coefficients theta",
                               leave=False):
            theta0 = theta.copy()
            theta0[k, :] = 0
            mat0 = np.zeros((dim2 - np.max(time_lags), rank))
            for L in range(d):
                mat0 += np.matmul(
                    X[np.max(time_lags) - time_lags[L]:dim2 - time_lags[L], :],
                    np.diag(theta0[L, :]))
            VarPi = X[np.max(time_lags):dim2, :] - mat0
            var1 = np.zeros((rank, rank))
            var2 = np.zeros(rank)
            for t in range(np.max(time_lags), dim2):
                B = X[t - time_lags[k], :]
                var1 += np.diag(np.multiply(B, B))
                var2 += np.matmul(np.diag(B), VarPi[t - np.max(time_lags), :])
            theta[k, :] = np.matmul(
                inv(var1 + lambda_theta * np.eye(rank) / lambda_x), var2)

        # Matrice imputée
        mat_hat = np.matmul(W, X.T)

        # Performance du modèle
        mape = np.sum(
            np.abs(dense_mat[pos_test] - mat_hat[pos_test]) /
            dense_mat[pos_test]) / dense_mat[pos_test].shape[0]
        rmse = np.sqrt(
            np.sum((dense_mat[pos_test] - mat_hat[pos_test])**2) /
            dense_mat[pos_test].shape[0])

        # Mise en liste pour la visualisation
        lmape.append(mape)
        lrmse.append(rmse)

        if (it + 1) % 50 == 0:  #200 == 0:
            print(('Iter: {}'.format(it + 1)))
            print(('Imputation MAPE: {:}'.format(mape)))
            print(('Imputation RMSE: {:}'.format(rmse)))
            print()

        # Si le rmse ne diminue pas, arrêter l'entrainement
        if lrmse[len(lrmse) - 1] < rmse:
            rmse = lrmse[len(lrmse) - 1]  # Sauvegarder le dernier RMSE
            break
            print("Le RMSE ne diminue pas. Arrêt de l'entrainement.")

    return mat_hat, mape, rmse, lmape, lrmse

## Entrainement du modèle

In [62]:

rank = list(range(1, 81, 16))

## Set hyparameters
lambda_w = list(range(1, 500, 250))
lambda_x = list(range(1, 500, 250))
lambda_theta = list(range(1, 500, 250))
eta = list(range(1, 11, 5))

time_lags = np.array([1, 2, 144]) # D'ordre 1
d = time_lags.shape[0]

maxiter = 10000

dim1, dim2 = sparse_mat.shape

entrainement = list()

for r in notebook.tqdm(rank, desc="rang"):
    for e in notebook.tqdm(eta, desc="apprentissage", leave=False):
        for w in notebook.tqdm(lambda_w, desc="lambda_w", leave=False):
            for x in notebook.tqdm(lambda_x, desc="lambda_x", leave=False):
                for t in notebook.tqdm(lambda_theta,
                                       desc="lambda_theta",
                                       leave=False):

                    # Formation des données en entrée
                    np.random.seed(2020) # Reproductibilité
                    W = 0.1 * np.random.rand(dim1, r)
                    np.random.seed(2020) # Reproductibilité
                    X = 0.1 * np.random.rand(dim2, r)
                    theta = 0.1 * np.random.rand(d, r)
                    init_para = {"W": W, "X": X, "theta": theta}

                    init_hyper = {
                        "lambda_w": w,
                        "lambda_x": x,
                        "lambda_theta": t,
                        "eta": e / 10
                    }

                    mat_hat, mape, rmse, lmape, lrmse = TRMF(
                        dense_mat, sparse_mat, init_para, init_hyper,
                        time_lags, maxiter)
                    print(mape, rmse)
                    entrainement.append([mape, rmse, init_hyper])

Iter: 50
Imputation MAPE: 0.14117432051656237
Imputation RMSE: 5.4478654928945955



Iter: 100
Imputation MAPE: 0.14117654188023113
Imputation RMSE: 5.447798469325509



Iter: 150
Imputation MAPE: 0.14117731366106284
Imputation RMSE: 5.447775522620083



Iter: 200
Imputation MAPE: 0.14117769667973398
Imputation RMSE: 5.4477635593407125



Iter: 250
Imputation MAPE: 0.141177920280548
Imputation RMSE: 5.447755983786612



...

Iter: 1900
Imputation MAPE: 0.14117805072773365
Imputation RMSE: 5.447700142105889



Iter: 1950
Imputation MAPE: 0.14117803172082738
Imputation RMSE: 5.447699053734797



Iter: 2000
Imputation MAPE: 0.14117801254141685
Imputation RMSE: 5.4476979698414905

0.14117801254141685 5.4476979698414905


Iter: 50
Imputation MAPE: 0.14117433202680843
Imputation RMSE: 5.447866192359501



KeyboardInterrupt: 

In [59]:
# Trouver les paramètres avec le plus petit erreur d'entrainement
entrainement_df = pd.DataFrame(entrainement, columns = ["mape", "rmse", "hyper"])
min_id = entrainement_df[["rmse"]].idxmin() 
entrainement_df.iloc[min_id]

,mape,rmse,hyper
0,0.141178,5.447764,"{'lambda_w': 1, 'lambda_x': 1, 'lambda_theta':..."


# Présentation des résultats

In [8]:
print('Imputation MAPE final: {:.6}'.format(mape))
print('Imputation RMSE final: {:.6}'.format(rmse))

Imputation MAPE final: 0.144843
Imputation RMSE final: 5.7168


In [9]:
lmape

[0.14630192002604292, 0.14484266542723576]

In [10]:
lrmse

[5.69934176049316, 5.716796802637134]